# Step 1: Login as External Researcher

In [ ]:
# syft absolute
import syft as sy

datasites = []
for i in range(3):
    server = sy.orchestra.launch(
        name="fl-datasite-" + str(i)
    )  # connects to same server
    client = server.login(email="sheldon@caltech.edu", password="changethis")
    datasites.append(client)

# Step 2: Train models on all datasites

In [ ]:
results = []

for dasite in datasites:
    print()
    print(dasite)
    images_ptr = dasite.datasets["MNIST Dataset"].assets["images"]
    labels_ptr = dasite.datasets["MNIST Dataset"].assets["labels"]

    results.append(
        dasite.code.train(mnist_images=images_ptr, mnist_labels=labels_ptr).get_from(
            dasite
        )
    )

# Step 3: Merge models

In [ ]:
# stdlib
from collections import OrderedDict
from itertools import groupby


def ave(d):
    _data = sorted([i for b in d for i in b.items()], key=lambda x: x[0])
    _d = [(a, [j for _, j in b]) for a, b in groupby(_data, key=lambda x: x[0])]
    return OrderedDict(
        {a: ave(b) if isinstance(b[0], dict) else sum(b) / float(len(b)) for a, b in _d}
    )

In [ ]:
models_weights = [(lambda x: x[1])(x) for x in results]

In [ ]:
new_model_weights = ave(models_weights)

In [ ]:
# third party
import torch
from torch import nn

# Define the neural network class


class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 10)

    def forward(self, x):
        x = torch.log_softmax(self.fc1(x.view(-1, 784)), dim=1)
        return x

In [ ]:
new_model = MLP()
new_model.load_state_dict(new_model_weights)